## Legacy Code Maintainence

### Reanalysis Animation

In [ ]:
def equiv_theta_cases_animation_generator(equiv_theta, lat, pressure):
    fig, axes = plt.subplots(
        nrows=2, ncols=5, figsize=(32, 18), dpi=160, sharex=True, sharey=True
    )
    plt.rcParams.update({"font.size": 28})

    def init():
        return None

    def run(index):
        for (i, j), ax in np.ndenumerate(axes):
            ax.cla()
            match i:
                case 0:
                    ax.invert_yaxis()
                    tmp_year = list(monsoon_onset_sorted.keys())[j]
                    tmp_date = list(monsoon_onset_sorted.values())[j]
                case 1:
                    tmp_year = list(monsoon_onset_sorted.keys())[j - 5]
                    tmp_date = list(monsoon_onset_sorted.values())[j - 5]
            tmp = equiv_theta[tmp_year - 1979, index + 100]
            ax.contourf(
                lat,
                pressure,
                tmp,
                levels=np.linspace(320, 350, 16),
                extend="both",
                cmap="RdBu_r",
            )
            ax.contour(lat, pressure, tmp, levels=np.linspace(320, 350, 16), colors="k")

            ax.set_title(f"({tmp_year}, {tmp_date})")
        fig.suptitle(f"Calendar date = {index + 100}")
        return None

    ani = animation.FuncAnimation(fig, run, frames=70, interval=20, init_func=init)
    return ani

In [ ]:
def equiv_theta_and_streamfunction_composites_animation_generator(
    equiv_theta, streamfunction, equiv_theta_grids, streamfunction_grids
):
    fig, ax = plt.subplots(
        nrows=1, ncols=1, figsize=(16, 9), dpi=160, sharex=True, sharey=True
    )
    plt.rcParams.update({"font.size": 28})

    def init():
        return None

    def run(index):
        ax.cla()
        ax.invert_yaxis()
        tmp1 = np.mean(
            equiv_theta[
                np.array(list(monsoon_onset_sorted.keys())[:5]) - 1979,
                index + 100,
            ],
            axis=0,
        )
        tmp2 = np.mean(
            streamfunction[
                np.array(list(monsoon_onset_sorted.keys())[:5]) - 1979,
                index + 100,
            ],
            axis=0,
        )
        ax.contourf(
            equiv_theta_grids[1],
            equiv_theta_grids[0],
            tmp1,
            levels=np.linspace(320, 350, 16),
            extend="both",
            cmap="RdBu_r",
        )
        ax.contour(
            streamfunction_grids[1],
            streamfunction_grids[0],
            tmp2,
            levels=np.linspace(-3e10, 3e10, 16),
            colors="k",
        )
        ax.set_xlabel("latitude")
        ax.set_ylabel("pressure (Pa)")
        fig.suptitle(
            f"Contour - streamfunction\nShading - equiv_theta\nCalendar date = {index + 100}"
        )
        plt.tight_layout()
        return None

    ani = animation.FuncAnimation(fig, run, frames=70, interval=100, init_func=init)
    return ani

In [ ]:
def display_potential_temperature_composites_evolution(
    potential_temperature_composite: tuple[np.ndarray, np.ndarray],
    grids: dict[str, np.ndarray],
    start: int = 1,
    step: int = 1,
) -> matplotlib.animation.FuncAnimation:
    """
    Display the evolution of potential temperature composites as an animation.

    Parameters:
    ----------
    potential_temperature_composite : tuple of np.ndarray
        A tuple containing two arrays:
        - early_composite: the potential temperature data for early onset.
        - late_composite: the potential temperature data for late onset.
    grids : dict of np.ndarray
        A dictionary containing grid data with keys "lat" (latitude) and "plev" (pressure level).
    start : int, optional
        The starting index for the animation frames (default is 1).
    step : int, optional
        The step size for frames in the animation (default is 1).

    Returns:
    -------
    matplotlib.animation.FuncAnimation
        An animation object showing the evolution of potential temperature composites.
    """
    # Unpack the composites
    early_composite, late_composite = potential_temperature_composite

    # Turn off interactive mode and set global font size for plots
    plt.ioff()
    plt.rcParams.update({"font.size": 28})

    # Create a figure with two subplots (early and late onset)
    figure, axes = plt.subplots(
        nrows=1, ncols=2, figsize=(16, 9), dpi=160, sharex=True, sharey=True
    )

    def init_animation():
        """Initialize the animation with an empty list (no initial content)."""
        return []

    def update_frame(days):
        """
        Update the animation frame by frame with early and late onset data.

        Parameters:
        ----------
        days : int
            The current day index for updating the frame.
        """
        # Define contour levels for potential temperature (theta)
        contour_levels = np.linspace(290, 350, 31)

        # Early Onset Composite Plot
        axes[0].cla()  # Clear previous content
        axes[0].invert_yaxis()  # Invert y-axis to have pressure levels top-down
        axes[0].set_title("Early Onset")
        axes[0].set_ylabel("Pressure (hPa)")
        axes[0].set_xlabel("Latitude")
        axes[0].contourf(
            grids["lat"],
            grids["plev"],
            early_composite[start + days],
            levels=contour_levels,
            extend="both",
            cmap="RdBu_r",
        )
        axes[0].contour(
            grids["lat"],
            grids["plev"],
            early_composite[start + days],
            levels=contour_levels,
            colors="k",
        )

        # Late Onset Composite Plot
        axes[1].cla()  # Clear previous content
        axes[1].set_title("Late Onset")
        axes[1].set_xlabel("Latitude")
        axes[1].contourf(
            grids["lat"],
            grids["plev"],
            late_composite[start + days],
            levels=contour_levels,
            extend="both",
            cmap="RdBu_r",
        )
        axes[1].contour(
            grids["lat"],
            grids["plev"],
            late_composite[start + days],
            levels=contour_levels,
            colors="k",
        )

        # Update the figure's overall title to reflect the current day
        figure.suptitle(f"Theta\nCalendar Date = {start + days}")
        plt.tight_layout()

        return []

    # Create the animation object with initialized frames
    animation_obj = animation.FuncAnimation(
        figure,
        update_frame,
        frames=step,
        interval=100,
        init_func=init_animation,
    )

    return animation_obj

In [ ]:
def display_equivalent_potential_temperature_composites_evolution(
    equivalent_potential_temperature_composite: tuple[np.ndarray, np.ndarray],
    grids: dict[str, np.ndarray],
    start: int = 1,
    step: int = 1,
) -> matplotlib.animation.FuncAnimation:
    """
    Display the evolution of potential temperature composites as an animation.

    Parameters:
    ----------
    potential_temperature_composite : tuple of np.ndarray
        A tuple containing two arrays:
        - early_composite: the potential temperature data for early onset.
        - late_composite: the potential temperature data for late onset.
    grids : dict of np.ndarray
        A dictionary containing grid data with keys "lat" (latitude) and "plev" (pressure level).
    start : int, optional
        The starting index for the animation frames (default is 1).
    step : int, optional
        The step size for frames in the animation (default is 1).

    Returns:
    -------
    matplotlib.animation.FuncAnimation
        An animation object showing the evolution of potential temperature composites.
    """
    # Unpack the composites
    early_composite, late_composite = equivalent_potential_temperature_composite

    # Turn off interactive mode and set global font size for plots
    plt.ioff()
    plt.rcParams.update({"font.size": 28})

    # Create a figure with two subplots (early and late onset)
    figure, axes = plt.subplots(
        nrows=1, ncols=2, figsize=(16, 9), dpi=160, sharex=True, sharey=True
    )

    def init_animation():
        """Initialize the animation with an empty list (no initial content)."""
        return []

    def update_frame(days):
        """
        Update the animation frame by frame with early and late onset data.

        Parameters:
        ----------
        days : int
            The current day index for updating the frame.
        """
        # Define contour levels for potential temperature (theta)
        contour_levels = np.linspace(300, 360, 31)

        # Early Onset Composite Plot
        axes[0].cla()  # Clear previous content
        axes[0].invert_yaxis()  # Invert y-axis to have pressure levels top-down
        axes[0].set_title("Early Onset")
        axes[0].set_ylabel("Pressure (hPa)")
        axes[0].set_xlabel("Latitude")
        axes[0].contourf(
            grids["lat"],
            grids["plev"],
            early_composite[start + days],
            levels=contour_levels,
            extend="both",
            cmap="RdBu_r",
        )
        axes[0].contour(
            grids["lat"],
            grids["plev"],
            early_composite[start + days],
            levels=contour_levels,
            colors="k",
        )

        # Late Onset Composite Plot
        axes[1].cla()  # Clear previous content
        axes[1].set_title("Late Onset")
        axes[1].set_xlabel("Latitude")
        axes[1].contourf(
            grids["lat"],
            grids["plev"],
            late_composite[start + days],
            levels=contour_levels,
            extend="both",
            cmap="RdBu_r",
        )
        axes[1].contour(
            grids["lat"],
            grids["plev"],
            late_composite[start + days],
            levels=contour_levels,
            colors="k",
        )

        # Update the figure's overall title to reflect the current day
        figure.suptitle(f"Theta\nCalendar Date = {start + days}")
        plt.tight_layout()

        return []

    # Create the animation object with initialized frames
    animation_obj = animation.FuncAnimation(
        figure,
        update_frame,
        frames=step,
        interval=100,
        init_func=init_animation,
    )

    return animation_obj

In [ ]:
def display_streamfunction_and_potential_temperature_composite_evolution(
    streamfunction_early_composite: np.ndarray,
    streamfunction_grids: dict[str, np.ndarray],
    potential_temperature_early_composite: np.ndarray,
    potential_temperature_grids: dict[str, np.ndarray],
    start: int = 1,
    step: int = 1,
) -> matplotlib.animation.FuncAnimation:
    """
    Display the evolution of streamfunction and potential temperature composites
    as an animation.

    Parameters:
    ----------
    streamfunction_early_composite : np.ndarray
        The streamfunction data for early onset.
    streamfunction_grids : dict of np.ndarray
        Grid data for the streamfunction with keys "lat" (latitude) and "plev" (pressure levels).
    potential_temperature_early_composite : np.ndarray
        The potential temperature data for early onset.
    potential_temperature_grids : dict of np.ndarray
        Grid data for potential temperature with keys "lat" (latitude) and "plev" (pressure levels).
    start : int, optional
        The starting index for the animation frames (default is 1).
    step : int, optional
        The step size for frames in the animation (default is 1).

    Returns:
    -------
    matplotlib.animation.FuncAnimation
        An animation object showing the evolution of streamfunction and equivalent potential temperature.
    """
    # Turn off interactive mode for plotting and set font size
    plt.ioff()
    plt.rcParams.update({"font.size": 26})

    # Create a single subplot for the animation
    figure, ax = plt.subplots(
        nrows=1, ncols=1, figsize=(16, 9), dpi=160, sharex=True, sharey=True
    )

    def init_animation():
        """Initialize the animation (no initial content)."""
        return []

    def update_frame(days):
        """
        Update the animation for each frame by plotting streamfunction and equivalent potential temperature.

        Parameters:
        ----------
        days : int
            The current day index for updating the frame.
        """
        # Clear previous plot
        ax.cla()
        ax.invert_yaxis()  # Invert y-axis to have pressure levels top-down

        # Define contour levels for potential temperature (theta) and streamfunction
        theta_levels = np.linspace(300, 330, 16)
        stream_levels = np.linspace(-2e10, 2e10, 16)

        # Plot potential temperature as filled contours
        ax.contourf(
            potential_temperature_grids["lat"],
            potential_temperature_grids["plev"],
            potential_temperature_early_composite[start + days],
            levels=theta_levels,
            extend="both",
            cmap="RdBu_r",
        )

        # Overlay streamfunction contours
        ax.contour(
            streamfunction_grids["lat"],
            streamfunction_grids["plev"],
            streamfunction_early_composite[start + days],
            levels=stream_levels,
            colors="k",
        )

        # Set labels for axes and title
        ax.set_xlabel("Latitude")
        ax.set_ylabel("Pressure (hPa)")
        figure.suptitle(
            f"Contour - Streamfunction\nShading - Potential Temperature\nCalendar Date = {start + days}"
        )
        plt.tight_layout()

        return []

    # Create the animation object
    animation_obj = animation.FuncAnimation(
        figure,
        update_frame,
        frames=step,
        interval=100,
        init_func=init_animation,
    )

    return animation_obj

In [ ]:
def display_streamfunction_and_equivalent_potential_temperature_composite_evolution(
    streamfunction_early_composite: np.ndarray,
    streamfunction_grids: dict[str, np.ndarray],
    equivalent_potential_temperature_early_composite: np.ndarray,
    equivalent_potential_temperature_grids: dict[str, np.ndarray],
    start: int = 1,
    step: int = 1,
) -> matplotlib.animation.FuncAnimation:
    """
    Display the evolution of streamfunction and equivalent potential temperature composites
    as an animation.

    Parameters:
    ----------
    streamfunction_early_composite : np.ndarray
        The streamfunction data for early onset.
    streamfunction_grids : dict of np.ndarray
        Grid data for the streamfunction with keys "lat" (latitude) and "plev" (pressure levels).
    equivalent_potential_temperature_early_composite : np.ndarray
        The equivalent potential temperature data for early onset.
    equivalent_potential_temperature_grids : dict of np.ndarray
        Grid data for equivalent potential temperature with keys "lat" (latitude) and "plev" (pressure levels).
    start : int, optional
        The starting index for the animation frames (default is 1).
    step : int, optional
        The step size for frames in the animation (default is 1).

    Returns:
    -------
    matplotlib.animation.FuncAnimation
        An animation object showing the evolution of streamfunction and equivalent potential temperature.
    """
    # Turn off interactive mode for plotting and set font size
    plt.ioff()
    plt.rcParams.update({"font.size": 26})

    # Create a single subplot for the animation
    figure, ax = plt.subplots(
        nrows=1, ncols=1, figsize=(16, 9), dpi=160, sharex=True, sharey=True
    )

    def init_animation():
        """Initialize the animation (no initial content)."""
        return []

    def update_frame(days):
        """
        Update the animation for each frame by plotting streamfunction and equivalent potential temperature.

        Parameters:
        ----------
        days : int
            The current day index for updating the frame.
        """
        # Clear previous plot
        ax.cla()
        ax.invert_yaxis()  # Invert y-axis to have pressure levels top-down

        # Define contour levels for equivalent potential temperature (theta) and streamfunction
        theta_levels = np.linspace(320, 350, 16)
        stream_levels = np.linspace(-2e10, 2e10, 16)

        # Plot equivalent potential temperature as filled contours
        ax.contourf(
            equivalent_potential_temperature_grids["lat"],
            equivalent_potential_temperature_grids["plev"],
            equivalent_potential_temperature_early_composite[start + days],
            levels=theta_levels,
            extend="both",
            cmap="RdBu_r",
        )

        # Overlay streamfunction contours
        ax.contour(
            streamfunction_grids["lat"],
            streamfunction_grids["plev"],
            streamfunction_early_composite[start + days],
            levels=stream_levels,
            colors="k",
        )

        # Set labels for axes and title
        ax.set_xlabel("Latitude")
        ax.set_ylabel("Pressure (hPa)")
        figure.suptitle(
            f"Contour - Streamfunction\nShading - Equivalent Potential Temperature\nCalendar Date = {start + days}"
        )
        plt.tight_layout()

        return []

    # Create the animation object
    animation_obj = animation.FuncAnimation(
        figure,
        update_frame,
        frames=step,
        interval=100,
        init_func=init_animation,
    )

    return animation_obj

In [ ]:
def display_mse_flux_divergence_composites_evolution(
    mse_flux_divergence_composite: tuple[np.ndarray, np.ndarray],
    grids: dict[str, np.ndarray],
    start: int = 1,
    step: int = 1,
) -> matplotlib.animation.FuncAnimation:
    """
    Display the evolution of MSE flux divergence composites as an animation.

    Parameters:
    ----------
    mse_flux_composite : tuple of np.ndarray
        A tuple containing two arrays:
        - early_composite: the MSE flux divergence data for early onset.
        - late_composite: the MSE flux divergence data for late onset.
    grids : dict of np.ndarray
        Grid data for MSE flux divergence with keys "lat" (latitude) and "plev" (pressure levels).
    start : int, optional
        The starting index for the animation frames (default is 1).
    step : int, optional
        The step size for frames in the animation (default is 1).

    Returns:
    -------
    matplotlib.animation.FuncAnimation
        An animation object showing the evolution of MSE flux composites.
    """
    early_composite, late_composite = mse_flux_divergence_composite

    # Turn off interactive mode and set global font size for plots
    plt.ioff()
    plt.rcParams.update({"font.size": 28})

    # Create a figure with two subplots for early and late onset composites
    figure, axes = plt.subplots(
        nrows=1, ncols=2, figsize=(16, 9), dpi=160, sharex=True, sharey=True
    )

    def init_animation():
        """Initialize the animation (no initial content)."""
        return []

    def update_frame(days):
        """
        Update the animation frame by frame, plotting the MSE flux composites.

        Parameters:
        ----------
        days : int
            The current day index for updating the frame.
        """
        # Define contour levels for MSE flux
        contour_levels = np.linspace(-1, 1, 16)

        # Plot Early Onset Composite (left subplot)
        axes[0].cla()  # Clear the previous content
        axes[0].invert_yaxis()  # Invert y-axis to have pressure levels top-down
        axes[0].set_title("Early Onset")
        axes[0].contourf(
            grids["lat"],
            grids["plev"],
            early_composite[start + days],
            levels=contour_levels,
            extend="both",
            cmap="RdBu_r",
        )
        axes[0].contour(
            grids["lat"],
            grids["plev"],
            early_composite[start + days],
            levels=contour_levels,
            colors="k",
        )
        axes[0].set_xlabel("Latitude")
        axes[0].set_ylabel("Pressure (hPa)")

        # Plot Late Onset Composite (right subplot)
        axes[1].cla()  # Clear the previous content
        axes[1].set_title("Late Onset")
        axes[1].contourf(
            grids["lat"],
            grids["plev"],
            late_composite[start + days],
            levels=contour_levels,
            extend="both",
            cmap="RdBu_r",
        )
        axes[1].contour(
            grids["lat"],
            grids["plev"],
            late_composite[start + days],
            levels=contour_levels,
            colors="k",
        )
        axes[1].set_xlabel("Latitude")

        # Update the figure's overall title with the current day
        figure.suptitle(f"MSE Flux Vertical Divergence\nCalendar Date = {start + days}")
        plt.tight_layout()

        return []

    # Create the animation object
    animation_obj = animation.FuncAnimation(
        figure,
        update_frame,
        frames=step,
        interval=100,
        init_func=init_animation,
    )

    return animation_obj

### Power Spectrum Plot

In [ ]:
def plot_spectral_contour(
    PSD_data,
    spectral_grid,
    wavenumber_range,
    title,
    xlabel,
    ylabel,
    log_value=True,
):
    """Helper function to plot a spectral contour with filled levels and contours."""
    wavenumbers, positive_frequencies = spectral_grid[2], spectral_grid[0]
    plt.rcParams.update({"font.size": 22})

    plt.figure(figsize=(16, 9), dpi=160)

    wavenum_indices = slice(
        np.argmax(wavenumbers >= wavenumber_range[0]),
        np.argmax(wavenumbers >= wavenumber_range[1]) + 1,
    )
    if log_value:
        plt.contourf(
            wavenumbers[wavenum_indices],
            positive_frequencies,
            np.log10(PSD_data[:, wavenum_indices]),
            levels=16,
            cmap="Greys",
        )
        plt.contour(
            wavenumbers[wavenum_indices],
            positive_frequencies,
            np.log10(PSD_data[:, wavenum_indices]),
            levels=32,
            colors="black",
        )
    else:
        plt.contourf(
            wavenumbers[wavenum_indices],
            positive_frequencies,
            PSD_data[:, wavenum_indices],
            levels=16,
            cmap="Greys",
            extend="min",
        )
        plt.contour(
            wavenumbers[wavenum_indices],
            positive_frequencies,
            PSD_data[:, wavenum_indices],
            levels=32,
            colors="black",
        )
    plt.plot([0, 0], [0, positive_frequencies[-1]], "k--", lw=2, zorder=20)
    plt.gca().add_patch(
        Rectangle(
            (wavenumber_range[0], positive_frequencies[0]),
            wavenumber_range[1] - wavenumber_range[0],
            positive_frequencies[1] - positive_frequencies[0],
            edgecolor="white",
            facecolor="white",
            fill=True,
            lw=4,
            zorder=10,
        )
    )

    plt.gca().add_patch(
        Rectangle(
            (wavenumber_range[0], positive_frequencies[-2]),
            wavenumber_range[1] - wavenumber_range[0],
            positive_frequencies[1] - positive_frequencies[0],
            edgecolor="white",
            facecolor="white",
            fill=True,
            lw=4,
            zorder=10,
        )
    )

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)


def OLR_PSD_plot(olr_PSD, olr_spectral_grid, type="Undefined"):
    olr_positive_PSD, positive_spectral_grid = extract_positive_PSD(
        olr_PSD, olr_spectral_grid
    )

    plot_spectral_contour(
        olr_positive_PSD,
        positive_spectral_grid,
        wavenumber_range=(-50, 50),
        title=f"OLR {type} Log Power Spectrum",
        xlabel="wavenumbers",
        ylabel="frequencies (cpd)",
    )
    return 0


def OLR_background_PSD_plot(olr_PSD, olr_spectral_grid):
    positive_background_PSD, positive_spectral_grid = extract_positive_PSD(
        olr_PSD, olr_spectral_grid
    )

    plot_spectral_contour(
        positive_background_PSD,
        positive_spectral_grid,
        wavenumber_range=(-15, 15),
        title="OLR Background Log Power Spectrum",
        xlabel="wavenumbers",
        ylabel="frequencies (cpd)",
    )
    return 0


def OLR_PSD_ratio_plot(
    olr_PSD, olr_background_PSD, olr_spectral_grid, var="OLR", type="Undefined"
):
    positive_PSD, positive_spectral_grid = extract_positive_PSD(
        olr_PSD, olr_spectral_grid
    )
    positive_background_PSD, _ = extract_positive_PSD(
        olr_background_PSD, olr_spectral_grid
    )

    PSD_ratio = positive_PSD / positive_background_PSD

    plot_spectral_contour(
        PSD_ratio,
        positive_spectral_grid,
        wavenumber_range=(-50, 50),
        title=f"{var} {type} Power Spectrum Ratio",
        xlabel="wavenumbers",
        ylabel="frequencies (cpd)",
        log_value=False,
    )
    return 0
